In [1]:
import pickle
import glob
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
df = pd.read_csv('/Users/zkq5md/Desktop/UVA/Data/SE_precip_data.csv', index_col='Unnamed: 0')
df['Year'] = pd.to_datetime(df['Datetimes']).dt.year
df['Month'] = pd.to_datetime(df['Datetimes']).dt.month
df['Day'] = pd.to_datetime(df['Datetimes']).dt.day
df

,Datetimes,Member,Maximum Precipitation,Max Value Lat,Min Value Lon,Average Precipitation,Year,Month,Day
0,1940-01-01 00:00:00,1,21.29,29.685864,268.75,3.80,1940,1,1
1,1940-01-02 00:00:00,1,30.17,34.397906,275.00,9.46,1940,1,2
2,1940-01-03 00:00:00,1,12.77,34.397906,280.00,1.67,1940,1,3
3,1940-01-04 00:00:00,1,3.27,30.628272,275.00,0.66,1940,1,4
4,1940-01-05 00:00:00,1,2.49,31.570681,280.00,0.11,1940,1,5
...,...,...,...,...,...,...,...,...,...
56625,2014-12-28 00:00:00,5,31.88,35.340314,270.00,6.95,2014,12,28
56626,2014-12-29 00:00:00,5,26.74,34.397906,272.50,9.22,2014,12,29
56627,2014-12-30 00:00:00,5,16.39,35.340314,276.25,4.05,2014,12,30
56628,2014-12-31 00:00:00,5,19.74,30.628272,266.25,5.47,2014,12,31


In [3]:
#create random testing and training datasets
df_train = df.sample(frac=0.7).copy()
df.drop(df_train.index.values, inplace=True)
df_train.to_csv('/Users/zkq5md/Desktop/UVA/Data/Training/Model_Datasets/df_train.csv')

df_val = df.sample(frac=0.33).copy()
df.drop(df_val.index.values, inplace=True)
df_val.to_csv('/Users/zkq5md/Desktop/UVA/Data/Training/Model_Datasets/df_val.csv')

df_test = df
df_test.to_csv('/Users/zkq5md/Desktop/UVA/Data/Training/Model_Datasets/df_test.csv')

print(56630-len(df_train)-len(df_val)-len(df_test))

0


In [4]:
ds = xr.open_dataset('/Users/zkq5md/Desktop/UVA/Data/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h1.PRECT.19400101-19491231.nc')
ds

<xarray.Dataset>
Dimensions:       (lat: 192, lon: 288, lev: 32, ilev: 33, time: 3650, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) object 1940-01-01 00:00:00 ... 1949-12-31 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/26)
    gw            (lat) float64 ...
    hyam          (lev) float64 ...
    hybm          (lev) float64 ...
    P0            float64 ...
    hyai          (ilev) float64 ...
    hybi          (ilev) float64 ...
    ...            ...
    n2ovmr        (time) float64 ...
    f11vmr        (time) float64 ...
    f12vmr        (time) float64 ...
    sol_tsi       (time) float64 ...
    nsteph        (time) float64 ...
    PRECT         (time, lat, lon) float32 ...
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTcmip6.f09_g17.LE2-1001.001
    logname:           sunseon
    host:              mom2
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1001-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1

In [5]:
data_path = '/Users/zkq5md/Desktop/UVA/Data/Training/'

for dataset_name in ['train', 'val', 'test']:

    df = locals()[f'df_{dataset_name}']
    
    image_data = []
    max_precip = []
    avg_precip = []
    i=0
    
    print(dataset_name)

    for did, row in df.iterrows():
        
        cwv = f'{data_path}/CWV/CWV_{row.Year}_{str(row.Month).zfill(2)}_{str(row.Day).zfill(2)}_mem{row.Member}.pkl'
        cwv_data = np.load(cwv, allow_pickle=True)

        z500= f'{data_path}/Z500/Z500_{row.Year}_{str(row.Month).zfill(2)}_{str(row.Day).zfill(2)}_mem{row.Member}.pkl'
        z500_data = np.load(z500, allow_pickle=True)

        temp= f'{data_path}/Temps/temps_{row.Year}_{str(row.Month).zfill(2)}_{str(row.Day).zfill(2)}_mem{row.Member}.pkl'
        temp_data= np.load(temp, allow_pickle=True)

        image_data.append([cwv_data[119:149, 188:237], z500_data[119:149, 188:237], temp_data[119:149, 188:237]])


        print((i/len(df)), end="\r")
        i = i+1

    max_precip.append(df['Maximum Precipitation'].values)
    avg_precip.append(df['Average Precipitation'].values)
                    
    pickle.dump(image_data, open(f'{data_path}Model_Datasets/X_{dataset_name}.pkl','wb'))
    pickle.dump(max_precip, open(f'{data_path}Model_Datasets/Y_maxprecip_{dataset_name}.pkl','wb'))
    pickle.dump(avg_precip, open(f'{data_path}Model_Datasets/Y_avgprecip_{dataset_name}.pkl','wb'))
        

train
val9997477359299717544
test982161969318583443
